In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os.path
import pickle
import cv2
import keras
import imutils
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.core import Flatten, Dense
from keras.layers.convolutional import MaxPooling2D, Conv2D

Using TensorFlow backend.


In [2]:
chars_image_folder = 'extract_number_images'
model_filename = 'phone_number_model.hdf5'
model_labels_filename = 'model_labels.dat'

In [3]:
data = []
labels = []

因為每個數字圖檔的尺寸並不完全相同，所以需要先重新調整成相同尺寸才能建立資料集

In [4]:
def resize_to_fit(image, width, height):
    padW = int((width - image.shape[1]) / 2.0)
    padH = int((height - image.shape[0]) / 2.0)
    image = cv2.copyMakeBorder(image, padH, padH, padW, padW,
        cv2.BORDER_REPLICATE)
    image = cv2.resize(image, (width, height))
    return image

讀取所有檔案，並將圖檔先重新調整尺寸為12*12後再建立資料集

In [6]:
for image_file in paths.list_images(chars_image_folder):
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = resize_to_fit(image, 12,12)
    image = np.expand_dims(image, axis = 2)
    label = image_file.split(os.path.sep)[-2]
    data.append(image)
    labels.append(label)

將資料分割成訓練組與測試組，並將數字轉成0~1之間的數值

In [7]:
data = np.array(data, dtype = 'float') / 255.0
labels = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.3)

In [8]:
lb = LabelBinarizer().fit(y_train)
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)

將標籤另存成一個檔案，後續預測時可以從這個標籤檔案中讀取標籤並轉換成實際預測的數字而不是機率

In [9]:
with open(model_labels_filename, 'wb') as f:
    pickle.dump(lb, f)

建立CNN模型，先進行兩次卷積，再建立一個三層fully-connected網路，最後輸出11個類別的特徵(0、1、2、3、4、5、6、7、8、9、#)

In [10]:
model = Sequential()

model.add(Conv2D(50,(5,5), padding = 'same', input_shape = (12,12,1), activation = 'relu'))
model.add(Conv2D(50,(5,5), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size= (2,2), strides = (2,2)))

model.add(Conv2D(50, (5,5), padding = 'same', activation = 'relu'))
model.add(Conv2D(50, (5,5), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size= (2,2), strides = (2,2)))

model.add(Flatten())
model.add(Dense(500, activation = 'relu'))
model.add(Dense(500, activation = 'relu'))
model.add(Dense(500, activation = 'relu'))
model.add(Dense(11, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam', metrics = ['accuracy'])

第一次訓練模型時，使用50個epochs進行訓練，得到預測準確率為1的模型，不過實際測試只需要10個epochs就可以達到相同效果

In [12]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 10, epochs = 10, verbose = 1)

Train on 5287 samples, validate on 2266 samples
Epoch 1/10
5287/5287 [==============================] - 55s - loss: 1.2538e-07 - acc: 1.0000 - val_loss: 1.2076e-07 - val_acc: 1.0000
Epoch 2/10
5287/5287 [==============================] - 50s - loss: 1.2343e-07 - acc: 1.0000 - val_loss: 1.2018e-07 - val_acc: 1.0000
Epoch 3/10
5287/5287 [==============================] - 52s - loss: 1.2217e-07 - acc: 1.0000 - val_loss: 1.1987e-07 - val_acc: 1.0000
Epoch 4/10
5287/5287 [==============================] - 51s - loss: 1.2129e-07 - acc: 1.0000 - val_loss: 1.1974e-07 - val_acc: 1.0000
Epoch 5/10
5287/5287 [==============================] - 49s - loss: 1.2079e-07 - acc: 1.0000 - val_loss: 1.1952e-07 - val_acc: 1.0000
Epoch 6/10
5287/5287 [==============================] - 48s - loss: 1.2037e-07 - acc: 1.0000 - val_loss: 1.1945e-07 - val_acc: 1.0000
Epoch 7/10
5287/5287 [==============================] - 50s - loss: 1.2007e-07 - acc: 1.0000 - val_loss: 1.1937e-07 - val_acc: 1.0000
Epoch 8/10
528

In [34]:
model.save(model_filename)